# Gradient boosting

## Setup

In [1]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)

Windows-10-10.0.19045-SP0
Python 3.10.5 (tags/v3.10.5:f377153, Jun  6 2022, 16:14:13) [MSC v.1929 64 bit (AMD64)]


In [2]:
import os
import numpy as np
import pandas as pd

from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import train_test_split

In [3]:
# load data
train = pd.read_csv('../data/final/train.csv')
test = pd.read_csv('../data/final/test.csv')

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Expenditure,CabinNum,GroupSize,FamilySize,HomePlanet,CryoSleep,Destination,VIP,NoSpending,CabinDeck,CabinSide,Solo,Transported
1205,10.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,248.0,1.0,7.0,Mars,True,TRAPPIST-1e,False,True,F,S,True,True
1070,26.0,6.431331,0.000000,5.236442,0.000000,7.173958,7.655391,232.0,1.0,9.0,Earth,False,55 Cancri e,missing,False,F,P,True,False
5462,18.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,944.0,1.0,7.0,Earth,True,55 Cancri e,False,True,G,S,True,True
2829,13.0,6.361302,4.356709,0.000000,5.726848,1.791759,6.874198,579.0,1.0,7.0,Earth,False,TRAPPIST-1e,False,False,F,S,True,False
5075,26.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,164.0,1.0,1.0,Europa,True,55 Cancri e,False,True,D,S,True,True


In [4]:
TARGET = 'Transported'
FEATURES = [col for col in train.columns if col not in [TARGET]]

numerical = train[FEATURES].select_dtypes(include=np.number).columns.to_list()
categorical = train[FEATURES].select_dtypes(exclude=np.number).columns.to_list()

train[numerical] = train[numerical].astype(float)
train[categorical] = train[categorical].astype(str)
train[TARGET] = train[TARGET].astype(float)

test[numerical] = test[numerical].astype(float)
test[categorical] = test[categorical].astype(str)

print(f'Target: {TARGET}')
print(f'Features:\n\tnumerical: {numerical}\n\tcategorical:{categorical}')
print(f'Shapes:\n\ttrain: {train.shape}\n\ttest: {test.shape}')

train.sample(5)

Target: Transported
Features:
	numerical: ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Expenditure', 'CabinNum', 'GroupSize', 'FamilySize']
	categorical:['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'NoSpending', 'CabinDeck', 'CabinSide', 'Solo']
Shapes:
	train: (8693, 19)
	test: (4277, 18)


## Models

In [5]:
# data splitting
x, x_val, y, y_val = train_test_split(
    train[FEATURES],
    train[TARGET],
    test_size = 0.2,
    random_state = 42
)

# pool preparation
pool_train = Pool(x, y, cat_features=categorical)
pool_val = Pool(x_val, y_val, cat_features=categorical)

In [6]:
# default 
model = CatBoostClassifier(
    iterations = 1500,
    learning_rate=0.03,
    custom_metric = ['Accuracy', 'AUC', 'F1'],
    allow_writing_files=False
)

_ = model.fit(
    pool_train,
    eval_set = pool_val,
    verbose = 250
)

0:	learn: 0.6821451	test: 0.6823971	best: 0.6823971 (0)	total: 244ms	remaining: 6m 6s
250:	learn: 0.3704423	test: 0.4016457	best: 0.4016457 (250)	total: 18s	remaining: 1m 29s
500:	learn: 0.3301812	test: 0.3867284	best: 0.3866900 (497)	total: 38.5s	remaining: 1m 16s
750:	learn: 0.3004252	test: 0.3838581	best: 0.3838504 (738)	total: 56.1s	remaining: 55.9s
1000:	learn: 0.2739637	test: 0.3812347	best: 0.3811261 (968)	total: 1m 14s	remaining: 37.3s
1250:	learn: 0.2514862	test: 0.3829864	best: 0.3810011 (1007)	total: 1m 33s	remaining: 18.6s
1499:	learn: 0.2327021	test: 0.3848780	best: 0.3810011 (1007)	total: 1m 51s	remaining: 0us

bestTest = 0.3810011224
bestIteration = 1007

Shrink model to first 1008 iterations.


In [7]:
# prediction
pool_test = Pool(test[FEATURES], cat_features=categorical)
preds = model.predict(pool_test)

## Submission

In [8]:
sub = pd.read_csv('../data/raw/sample_submission.csv')
sub[TARGET] = preds.astype(bool)
sub

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [9]:
os.makedirs('../submissions', exist_ok=True)
sub.to_csv('../submissions/catboost_default.csv', index=False)